教程地址 https://conda.io/projects/conda/en/latest/

# Concepts
+ Conda commands
+ Conda packages
+ Conda package specification
+ Conda channels
+ Conda environments
+ Installing with conda
+ Conda performance
+ Conda for data scientists

## Conda commands
作用：
+ conda命令可以查询并搜索Anaconda包索引和当前Anaconda安装
+ 创建新的conda环境
+ 在现有的conda环境中安装和更新包  

> Tips:  
> You can abbreviate many frequently used command options that are preceded by 2 dashes (--) to just 1 dash and the first letter of the option. So --name and -n are the same, and --envs and -e are the same.

command reference https://conda.io/projects/conda/en/latest/commands.html  
Viewing command-line help https://conda.io/projects/conda/en/latest/user-guide/tasks/view-command-line-help.html 

## Conda packages
### What is a conda package?
A conda package is a compressed tarball file (.tar.bz2) or .conda file that contains:
+ system-level libraries  
+ Python or other modules  
+ executable programs and other components  
+ metadata under the info/ directory  
+ a collection of files that are installed directly into an install prefix

### .conda file format
These compressed files can be significantly smaller than their bzip2 equivalents. In addition, they decompress much more quickly. .conda is the preferred file format to use where available, although we continue to provide .tar.bz2 files in tandem.
> Note  
> In conda 4.7 and later, you cannot use package names that end in “.conda” as they conflict with the .conda file format for packages.

### Using packages
+ You may search for packages
      conda search scipy
+ You may install a packages
      conda install scipy
+ You may build a package after installing conda build
      conda build my_fun_package
   

### Package structure
    .
    ├── bin
    │   └── pyflakes
    ├── info
    │   ├── LICENSE.txt
    │   ├── files
    │   ├── index.json
    │   ├── paths.json
    │   └── recipe
    └── lib
        └── python3.5

+ bin contains relevant binaries for the package
+ lib contains the relevant library files (eg. the .py files)
+ info contains package metadata

### Metapackages
当conda包仅用于元数据不含任何文件时，它被称为元包  
An example of a metapackage is "anaconda," which collects together all the packages in the Anaconda installer. The command conda create -n envname anaconda creates an environment that exactly matches what would be created from the Anaconda installer. You can create metapackages with the conda metapackage command. Include the name and version in the command.  
*Anaconda metapackage*  
The Anaconda metapackage is used in the creation of the Anaconda Distribution installers so that they have a set of packages associated with them.  
read more https://www.anaconda.com/whats-in-a-name-clarifying-the-anaconda-metapackage/